In [5]:
from collections import Counter
from datasets import load_dataset,load_from_disk
dataset=load_from_disk('codeforces_cot_filtered_truncated')
dataset=dataset.filter(lambda ex:ex['language'] in ['python','cpp'])
langs = [ex["language"] for ex in dataset]
print(Counter(langs))


Filter:   0%|          | 0/54125 [00:00<?, ? examples/s]

Counter({'cpp': 42192, 'python': 11915})


In [8]:
dataset.save_to_disk('codeforces_cot_filtered_truncated_v1')

Saving the dataset (0/7 shards):   0%|          | 0/54107 [00:00<?, ? examples/s]

In [27]:
from collections import Counter
from datasets import load_dataset,load_from_disk
dataset=load_from_disk('new_dataset_annotated_train')
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_train_v1')

dataset=load_from_disk('new_dataset_annotated_test')
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_val_v1')

Saving the dataset (0/2 shards):   0%|          | 0/16316 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [24]:
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)


Filter:   0%|          | 0/54107 [00:00<?, ? examples/s]

In [ ]:
load_dataset("haj1r/sphinxnautics-codeforces-cot-v3")['train']

FileNotFoundError: Directory haj1r/sphinxnautics-codeforces-cot-v3 not found

In [50]:
from datasets import load_dataset

dataset = load_dataset("haj1r/sphinxnautics-codeforces-cot-v3")


In [45]:
from datasets import load_from_disk, DatasetDict

dataset_combined = DatasetDict({
    "train": load_from_disk("codeforces_cot_filtered_truncated_matched_train_v1"),
    "validation": load_from_disk("codeforces_cot_filtered_truncated_matched_test_v1")
})


In [48]:
ds = load_from_disk("codeforces_cot_filtered_truncated_matched_train_v1")
ds.to_json("train.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

735679355

In [ ]:
dataset_combined.save_to_disk('codeforces_cot_filtered_32B-v2')

Saving the dataset (0/2 shards):   0%|          | 0/16316 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [ ]:
kl_temp=5          
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):       
    outputs = model.generate(input_ids=inputs[0:,:last_question_index].to(device),attention_mask=attention_mask[0:,:last_question_index].to(device),early_stopping=early_stopping, temperature=temp,pad_token_id=pad_token_id, top_p=0.9,top_k=50,min_length=min_len, max_new_tokens=max_new_tokens,return_dict_in_generate=True,output_scores=True,do_sample=do_sample)
#                        s_logits = outputs.logits[:, -1, :]
    new_inputs=outputs['sequences']#torch.cat([inputs[0:,:last_question_index],]
    s_device=model.device
    s_outputs = model(input_ids=new_inputs.to(s_device),attention_mask=attention_mask)
    s_logits=s_outputs.logits[:,last_question_index:]
    s_logp = F.log_softmax(s_logits / kl_temp, dim=-1)

with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.bfloat16):       
    t_device=teacher.device
    attention_mask = torch.ones_like(new_inputs, dtype=torch.long)
    t_outputs = teacher(input_ids=new_inputs.to(t_device),attention_mask=attention_mask)
    t_logits=t_outputs.logits[:,last_question_index:]
    t_logp = F.log_softmax(t_logits / kl_temp, dim=-1)
loss_kl = F.kl_div(s_logp, t_logp, reduction="batchmean", log_target=True) * (kl_temp **2)


In [7]:
len(dataset)

18365

In [25]:
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_test_v1')

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [29]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
model_name='Qwen/Qwen2.5-7B-Instruct'
chkpoint_path="checkpoints4/Qwen2.5-7B-Instruct/codeforces_cot_filtered_truncated_matched_v1/epoch_1/model_step_500.pt"
chkpoint_path="checkpoints3p/Qwen2.5-7B-Instruct/codeforces_cot_filtered_truncated_matched_v1/epoch_4/model_step_1000.pt"
access_token=''
model = AutoModelForCausalLM.from_pretrained(model_name,token=access_token,torch_dtype=torch.bfloat16,attn_implementation="flash_attention_2",device_map='cpu')
ckpt = torch.load(chkpoint_path, map_location='cpu')
model.load_state_dict(ckpt, strict=False)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)
if tokenizer.pad_token_id is None:
        # Add a new unique padding token
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

dataset_name=chkpoint_path.split('/')[2]
if chkpoint_path.split('/')[0][-1]=='4':
        dataset_name+='_RL'
elif chkpoint_path.split('/')[0][-1]=='p':
        dataset_name+='_ACL'
epoch_num=chkpoint_path.split('/')[3].split('_')[-1]
start_step=chkpoint_path.split('/')[4].split('_')[-1]
save_dir=f"{model_name.split('/')[-1]}_epoch_{epoch_num}_step_{start_step}_ft_{dataset_name}"
        
print(save_dir,model.device)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL cpu


In [30]:
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/tokenizer_config.json',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/special_tokens_map.json',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/chat_template.jinja',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/vocab.json',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/merges.txt',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/added_tokens.json',
 'Qwen2.5-7B-Instruct_epoch_4_step_1000.pt_ft_codeforces_cot_filtered_truncated_matched_v1_ACL/tokenizer.json')